# Paper-ready Figures: Franka and Robot Navigation Experiments

This notebook loads the saved analysis outputs (experiment_analysis.json and experiment_raw_data.pkl) produced by the Franka and Robot Navigation experiments and generates compact, publication-quality figures.

- Titles use human-readable academic naming (no underscores).
- Preset color theme and tight figure layouts suitable for papers.
- Figures are saved back into each experiment output directory.

In [8]:
# Imports and plotting theme
import os, json, pickle, glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['DejaVu Sans', 'Helvetica', 'Arial'],
    'font.size': 10,
    'axes.titlesize': 11,
    'axes.labelsize': 10,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 9,
    'figure.dpi': 300,
    'axes.linewidth': 0.8
})

# Compact palette for paper figures
# PAPER_PALETTE = ['#CFD8DC', "#A9D7F7", "#49A8E7", "#4a89cc"]
PAPER_PALETTE = ["#D7E3E7", "#A9D7F7", "#49A8E7", "#4a89cc"]  # blue, gold, green, coral
PAPER_PALETTE2 = ["#FFF9A6", "#FFD87D", "#FFA726", "#FF4800"]  # blue, gold, green, coral

AI_PALETTE_EXTENDED = {
    # 核心色系
    'main': '#607D8B',        # 主色调（科技灰蓝）

    # 辅助色分级（新增2级）
    'aux1': '#CFD8DC',        # 浅灰蓝（次级数据）
    'aux2': '#71b7c8',        # 更浅灰蓝（背景元素）
    'aux5': '#3a6fa6',        # 中灰蓝（网格/注释）
    'aux4': '#455A64',        # 深灰蓝（强调文本）
    'aux3': '#FFA726',        # 活力橙（警示/交互）

    # 中性色体系
    'neutral1': '#ECEFF1',    # 浅灰背景（适配深色模式）
    'neutral2': '#263238',    # 深灰文字（高对比度）
    'neutral3': '#B0BEC5'     # 中灰网格（原aux2复用）
}


sns.set_style('white')
sns.set_palette(PAPER_PALETTE)

In [9]:
# Utility: load analysis JSON and raw pickle (if present)
def load_analysis(output_dir):
    analysis_path = Path(output_dir) / 'experiment_analysis.json'
    raw_path = Path(output_dir) / 'experiment_raw_data.pkl'
    analysis = None
    raw = None
    if analysis_path.exists():
        with open(analysis_path, 'r', encoding='utf-8') as f:
            analysis = json.load(f)
    if raw_path.exists():
        try:
            with open(raw_path, 'rb') as f:
                raw = pickle.load(f)
        except Exception:
            raw = None
    return analysis, raw

def nice_name(method_key):
    method_key_remap = {
        "AVWBFO_MC_Linear": "AV-WBFO + MC",
        "AVWBFO_LHS_Linear": "AV-WBFO + LHS",
        "MPPI_MC_Linear": "MPPI + MC",
        "MPPI_LHS_Linear": "MPPI + LHS",
        "elair_barrier_nav_VanillaRL": "Vanilla RL",
        "elair_barrier_nav_AVWBFO_RL": "AV-WBFO w RL",
        "elair_barrier_nav_AVWBFO": "AV-WBFO w/o RL",
        "elair_barrier_nav_MPPI_RL": "MPPI w RL",
        "elair_barrier_nav_MPPI": "MPPI w/o RL",
    }
    # Replace underscores, common abbreviations to nicer titles
    return method_key_remap.get(method_key, method_key.replace('_', ' '))

In [10]:
# Plot helpers: compact bar with error, compact table and reward curves
def bar_with_error(ax, labels, means, stds, colors=None, ylabel='', title=''):
    x = np.arange(len(labels))
    if colors is None:
        colors = PAPER_PALETTE[:len(labels)]
    bars = ax.bar(x, means, yerr=stds, color=colors, capsize=4, linewidth=0.6, edgecolor='black', alpha=0.95)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=0, ha='center')
    ax.set_ylabel(ylabel)
    ax.set_title(title, pad=6)
    ax.set_axisbelow(True)
    ax.grid(axis='y', alpha=0.25)
    # Add compact labels
    for b, m, s in zip(bars, means, stds):
        h = b.get_height()
        ax.text(b.get_x() + b.get_width()/4*3 if s>0 else b.get_x() + b.get_width()/2, 
                h + max(0.01, 0.02 * max(1.0, h)), 
                f'{m:.2f}\n±{s:.2f}' if s > 0 else f'{m:.2f}',
                ha='center', va='bottom', fontsize=8)

def compact_summary_table(save_path, headers, rows, title=''):
    fig, ax = plt.subplots(figsize=(6, max(1.2, 0.25 * len(rows) + 0.8)))
    ax.axis('off')
    table = ax.table(cellText=rows, colLabels=headers, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1, 1.2)
    if title:
        ax.set_title(title, pad=8, fontsize=11)
    fig.tight_layout(pad=0.5)
    fig.savefig(save_path, bbox_inches='tight')
    plt.close(fig)

def reward_curves(save_path, reward_histories_by_method, labels, colors, title=''):
    fig, ax = plt.subplots(figsize=(6,3.2))
    for method in labels:
        histories = reward_histories_by_method.get(method, [])
        if not histories:
            continue
        # pad sequences to equal length
        max_len = max(len(h) for h in histories)
        padded = np.array([np.pad(h, (0, max_len - len(h)), mode='edge') for h in histories])
        mean = padded.mean(axis=0)
        std = padded.std(axis=0)
        idx = np.arange(len(mean))
        color = colors[labels.index(method) % len(colors)]
        ax.plot(idx, mean, color=color, lw=1.5, label=nice_name(method))
        ax.fill_between(idx, mean - std, mean + std, color=color, alpha=0.18)
    ax.set_xlabel('Step')
    ax.set_ylabel('Mean Step Reward')
    ax.set_title(title)
    ax.grid(alpha=0.2)
    ax.legend(frameon=False, ncol=2, fontsize=8)
    fig.tight_layout(pad=0.6)
    fig.savefig(save_path, bbox_inches='tight')
    plt.close(fig)

In [11]:
# Discover experiment folders (pick latest if multiple) and plot for each
def find_latest_dir(pattern):
    candidates = glob.glob(pattern)
    if not candidates:
        return None
    # choose newest modified
    candidates = sorted(candidates, key=lambda p: os.path.getmtime(p), reverse=True)
    return candidates[0]

franka_dir = find_latest_dir('./franka_noise_experiment_*')
robot_dir = find_latest_dir('./robot_nav_experiment_*')

franka_dir = "/home/user/CodeSpace/Python/PredictiveDiffusionPlanner_Dev/doc/records/20250827 Exp1NumSample_Cost/franka_noise_experiment_reach_backward_20250830"
robot_dir = "E:/CodeTestFile/Github-private-repo/master_prj/PredictiveDiffusionPlanner_Dev/doc/records/20250902 Exp2ElAirBarrierNav/new_rew2"

print('Franka dir:', franka_dir)
print('Robot nav dir:', robot_dir)

Franka dir: /home/user/CodeSpace/Python/PredictiveDiffusionPlanner_Dev/doc/records/20250827 Exp1NumSample_Cost/franka_noise_experiment_reach_backward_20250830
Robot nav dir: E:/CodeTestFile/Github-private-repo/master_prj/PredictiveDiffusionPlanner_Dev/doc/records/20250902 Exp2ElAirBarrierNav/new_rew2


In [12]:
# Generate publication-ready figures for Franka experiment (if found)
if franka_dir:
    analysis, raw = load_analysis(franka_dir)
    if analysis is None:
        print('No analysis JSON in', franka_dir)
    else:
        methods = list(analysis.keys())
        # Completion rate bar
        labels = [nice_name(m) for m in methods]
        means = [analysis[m]['summary']['completion_rates']['mean'] for m in methods]
        stds = [analysis[m]['summary']['completion_rates']['std'] for m in methods]
        fig_path = os.path.join(franka_dir, 'paper_completion_rate.svg')
        fig, ax = plt.subplots(figsize=(5.4,2.4))
        bar_with_error(ax, labels, means, stds, colors=PAPER_PALETTE, ylabel='Completion Rate', title='Task Completion Rate')
        ax.set_ylim(0, 1.1)
        # ax.plot([0, len(labels)-1], [1.0, 1.0], 'k--', lw=0.8, alpha=0.6)  # ideal line
        ax.grid(axis='y', alpha=0.7)
        fig.tight_layout(pad=0.2)
        fig.savefig(fig_path, bbox_inches='tight')
        plt.close(fig)

        # Mean steps bar
        means_s = [analysis[m]['summary']['completion_steps']['mean'] for m in methods]
        stds_s = [analysis[m]['summary']['completion_steps']['std'] for m in methods]
        fig_path2 = os.path.join(franka_dir, 'paper_mean_steps.svg')
        fig, ax = plt.subplots(figsize=(5.4,2.4))
        bar_with_error(ax, labels, means_s, stds_s, colors=PAPER_PALETTE, ylabel='Mean Steps', title='Mean Steps to Completion')
        ax.set_xticklabels(labels, rotation=0, ha='center')
        fig.tight_layout(pad=0.2)
        fig.savefig(fig_path2, bbox_inches='tight')
        plt.close(fig)

        # Reward curves
        reward_histories = {}
        for m in methods:
            rh = analysis[m].get('raw_data', {}).get('reward_histories', [])
            reward_histories[m] = rh
        reward_path = os.path.join(franka_dir, 'paper_reward_profiles.svg')
        reward_curves(reward_path, reward_histories, methods, PAPER_PALETTE, title='Reward Profile by Method')

        # Summary table
        headers = ['Method', 'Completion Rate', 'Mean Steps', 'Avg Reward', 'Opt Time (s)']
        rows = []
        for m in methods:
            s = analysis[m]['summary']
            rows.append([nice_name(m),
                         f"{s['completion_rates']['mean']:.3f} ± {s['completion_rates']['std']:.3f}",
                         f"{s['completion_steps']['mean']:.1f} ± {s['completion_steps']['std']:.1f}",
                         f"{s['average_rewards']['mean']:.3f} ± {s['average_rewards']['std']:.3f}",
                         f"{s['optimization_times']['mean']:.3f} ± {s['optimization_times']['std']:.3f}"])
        table_path = os.path.join(franka_dir, 'paper_summary_table.svg')
        compact_summary_table(table_path, headers, rows, title='Experimental Summary')

        print('Saved Franka paper figures to', franka_dir)

No analysis JSON in /home/user/CodeSpace/Python/PredictiveDiffusionPlanner_Dev/doc/records/20250827 Exp1NumSample_Cost/franka_noise_experiment_reach_backward_20250830


In [20]:
# Generate publication-ready figures for Robot Navigation experiment (if found)
if robot_dir:
    NavSuccPlaette1 = ["#49A8E7", "#005cbe"]  # blue, gold, green, coral
    NavSuccPlaette2 = [ "#FFA726", "#D27E00"]  # blue, gold, green, coral

    
    analysis, raw = load_analysis(robot_dir)
    if analysis is None:
        print('No analysis JSON in', robot_dir)
    else:
        methods = list(analysis.keys())
        labels = [nice_name(m) for m in methods]
        
        # Navigation success rate with dual y-axis for mean reward
        means = [analysis[m]['summary']['navigation_success_rate']['mean'] for m in methods]
        stds = [analysis[m]['summary']['navigation_success_rate']['std'] for m in methods]
        reward_means = [analysis[m]['summary']['average_rewards']['mean'] for m in methods]
        reward_stds = [analysis[m]['summary']['average_rewards']['std'] for m in methods]
        
        fig_path = os.path.join(robot_dir, 'paper_navigation_success.svg')
        fig, ax1 = plt.subplots(figsize=(8, 4))
        
        # Primary y-axis: Navigation Success Rate
        x = np.arange(len(labels))
        width = 0.35
        bars1 = ax1.bar(x - width/2, means, width, color=NavSuccPlaette1[0], 
                        linewidth=0.6, edgecolor='black', alpha=0.8,
                        label='Navigation Success Rate')
        ax1.set_xlabel('Trajectory Optimization Method')
        ax1.set_ylabel('Navigation Success Rate', color=NavSuccPlaette1[1])
        ax1.set_title('Navigation Success Rate and Mean Reward by Method')
        ax1.tick_params(axis='y', labelcolor=NavSuccPlaette1[1])
        ax1.set_ylim(0, 1.2)
        ax1.set_xticks(x)
        ax1.set_xticklabels(labels, rotation=0, ha='center')
        ax1.grid(axis='y', alpha=0.25)
        
        # Secondary y-axis: Mean Reward
        ax2 = ax1.twinx()
        bars2 = ax2.bar(x + width/2, reward_means, width, color=NavSuccPlaette2[0],
                        linewidth=0.6, edgecolor='black', alpha=0.7,
                        label='Mean Reward')
        ax2.set_ylabel('Mean Reward', color=NavSuccPlaette2[1])
        ax2.tick_params(axis='y', labelcolor=NavSuccPlaette2[1])
        
        # Add value labels on bars (without error bars)
        for bar, mean in zip(bars1, means):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{mean:.2f}', ha='center', va='bottom', fontsize=8)
        
        for bar, mean in zip(bars2, reward_means):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{mean:.3f}', ha='center', va='bottom', fontsize=8)
        
        # Combine legends
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', frameon=False, fontsize=9)
        
        fig.tight_layout(pad=0.6)
        fig.savefig(fig_path, bbox_inches='tight')
        plt.close(fig)

        # Completion steps - Double bar chart (Completed vs All Environments)
        completed_only_steps = []
        completed_only_stds = []
        all_envs_steps = []
        all_envs_stds = []
        
        for method in methods:
            # Get max_steps from the first result (assuming all have same max_steps)
            max_steps = None
            if raw and method in raw:
                for result in raw[method]:
                    if 'max_steps_allowed' in result:
                        max_steps = result['max_steps_allowed']
                        break
            if max_steps is None:
                max_steps = 300  # fallback default
            
            # Collect all individual completion steps and calculate for both scenarios
            method_completed_steps = []
            method_all_steps = []
            
            if raw and method in raw:
                for result in raw[method]:
                    # Extract individual completion steps from per_env_completion_steps if available
                    if 'per_env_completion_steps' in result:
                        completion_steps_data = [step for _, step in result['per_env_completion_steps']]
                        method_completed_steps.extend(completion_steps_data)
                        
                        # For all environments calculation, need to know total envs and successful ones
                        num_main_envs = result.get('num_main_envs', 20)  # fallback
                        num_completed = len(completion_steps_data)
                        num_not_completed = num_main_envs - num_completed
                        print("cp:", num_completed)
                        
                        # Add completed steps
                        method_all_steps.extend(completion_steps_data)
                        # Add max_steps for non-completed environments
                        method_all_steps.extend([max_steps] * num_not_completed)
                    
                    # Fallback: if only mean is available
                    elif 'mean_completion_steps' in result and result.get('navigation_success_rate', 0) > 0:
                        method_completed_steps.append(result['mean_completion_steps'])
                        
                        # Estimate all environments based on success rate
                        success_rate = result.get('navigation_success_rate', 0)
                        num_main_envs = result.get('num_main_envs', 20)
                        num_completed = int(success_rate * num_main_envs)
                        num_not_completed = num_main_envs - num_completed
                        
                        method_all_steps.extend([result['mean_completion_steps']] * num_completed)
                        method_all_steps.extend([max_steps] * num_not_completed)
            
            # Calculate statistics
            if method_completed_steps:
                completed_only_steps.append(float(np.mean(method_completed_steps)))
                completed_only_stds.append(float(np.std(method_completed_steps)) if len(method_completed_steps) > 1 else 0.0)
            else:
                completed_only_steps.append(0.0)
                completed_only_stds.append(0.0)
            
            if method_all_steps:
                all_envs_steps.append(float(np.mean(method_all_steps)))
                all_envs_stds.append(float(np.std(method_all_steps)) if len(method_all_steps) > 1 else 0.0)
            else:
                all_envs_steps.append(max_steps)
                all_envs_stds.append(0.0)

        # Create double bar chart
        fig_path2 = os.path.join(robot_dir, 'paper_completion_steps_comparison.svg')
        fig, ax = plt.subplots(figsize=(8, 4))
        
        # Set up bar positions
        x = np.arange(len(labels))
        width = 0.44  # width of bars
        
        # Create bars
        bars1 = ax.bar(x - width/2, completed_only_steps, width, yerr=completed_only_stds,
                       color=PAPER_PALETTE[2], capsize=4, alpha=0.8, 
                       label='Completed Environments Only', edgecolor='black', linewidth=0.6)
        
        bars2 = ax.bar(x + width/2, all_envs_steps, width, yerr=all_envs_stds,
                       color=PAPER_PALETTE2[2], capsize=4, alpha=0.8, 
                       label='All Environments (Failed = Max Steps)', edgecolor='black', linewidth=0.6)

        ax.set_ylabel('Mean Steps to Completion')
        ax.set_title('Task Completion Efficiency by Method')
        ax.set_xlabel('Trajectory Optimization Method')
        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=0, ha='center')
        ax.set_ylim(0, max(max(completed_only_steps + all_envs_steps) * 1.3, 50))
        ax.legend(frameon=False, fontsize=9, loc='upper right')
        ax.grid(axis='y', alpha=0.25)

        # Add value labels on bars
        for i, (bar1, bar2, steps1, std1, steps2, std2) in enumerate(zip(
            bars1, bars2, completed_only_steps, completed_only_stds, all_envs_steps, all_envs_stds)):
            
            # Labels for completed only bars
            height1 = bar1.get_height()
            if height1 > 0:  # Only show label if there were completions
                ax.text(bar1.get_x() + bar1.get_width()/2., height1 + std1 + 5,
                        f'{steps1:.1f}±{std1:.1f}', ha='center', va='bottom', fontsize=8)
            
            # Labels for all environments bars
            height2 = bar2.get_height()
            ax.text(bar2.get_x() + bar2.get_width()/2., height2 + std2 + 5,
                    f'{steps2:.1f}±{std2:.1f}', ha='center', va='bottom', fontsize=8)

        fig.tight_layout(pad=0.6)
        fig.savefig(fig_path2, bbox_inches='tight')
        plt.close(fig)

        # Compact summary table
        headers = ['Method', 'Success Rate', 'Mean Steps', 'Avg Reward', 'Opt Time (s)']
        rows = []
        for m in methods:
            s = analysis[m]['summary']
            rows.append([nice_name(m),
                         f"{s['navigation_success_rate']['mean']:.3f} ± {s['navigation_success_rate']['std']:.3f}",
                         f"{s['completion_steps']['mean']:.1f} ± {s['completion_steps']['std']:.1f}",
                         f"{s['average_rewards']['mean']:.3f} ± {s['average_rewards']['std']:.3f}",
                         f"{s['optimization_times']['mean']:.3f} ± {s['optimization_times']['std']:.3f}"])
        table_path = os.path.join(robot_dir, 'paper_summary_table.svg')
        compact_summary_table(table_path, headers, rows, title='Experimental Summary')

        # If raw reward histories available, plot reward profiles
        reward_histories = {}
        print(analysis[m].keys())
        for m in methods:
            rh = analysis[m].get('raw_data', {}).get('reward_histories', [])
            reward_histories[m] = rh
        reward_path = os.path.join(robot_dir, 'paper_reward_profiles.svg')
        reward_curves(reward_path, reward_histories, methods, PAPER_PALETTE, title='Reward Profile by Method')

        print('Saved Robot Navigation paper figures to', robot_dir)
        print('Generated double bar chart: paper_completion_steps_comparison.svg')

cp: 0
cp: 20
cp: 15
cp: 20
cp: 10
dict_keys(['summary'])
Saved Robot Navigation paper figures to E:/CodeTestFile/Github-private-repo/master_prj/PredictiveDiffusionPlanner_Dev/doc/records/20250902 Exp2ElAirBarrierNav/new_rew2
Generated double bar chart: paper_completion_steps_comparison.svg
dict_keys(['summary'])
Saved Robot Navigation paper figures to E:/CodeTestFile/Github-private-repo/master_prj/PredictiveDiffusionPlanner_Dev/doc/records/20250902 Exp2ElAirBarrierNav/new_rew2
Generated double bar chart: paper_completion_steps_comparison.svg


C:\Users\22059\AppData\Local\Temp\ipykernel_49240\3449140773.py:53: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(frameon=False, ncol=2, fontsize=8)


In [18]:
if robot_dir:
    analysis, raw = load_analysis(robot_dir)
    print("=== Completion Steps by Environment and Method ===\n")
    
    for method in methods:
        print(f"Method: {nice_name(method)}")
        print("-" * 50)
        
        if method in raw:
            for run_idx, result in enumerate(raw[method]):
                print(f"  Run {run_idx + 1}:")
                
                if 'per_env_completion_steps' in result:
                    env_steps = result['per_env_completion_steps']
                    print(f"    Completed environments ({len(env_steps)}/20):")
                    # for env_idx, steps in env_steps:
                    #     print(f"      Env {env_idx}: {steps} steps")
                    cp_step =[]
                    for env_idx, steps in env_steps:
                        cp_step.append(steps)
                    print(cp_step)
                    # Show which environments failed (if any)
                    completed_envs = {env_idx for env_idx, _ in env_steps}
                    all_envs = set(range(20))  # assuming 20 environments
                    failed_envs = all_envs - completed_envs
                    if failed_envs:
                        max_steps = result.get('max_steps_allowed', 300)
                        print(f"    Failed environments ({len(failed_envs)}/20): {sorted(failed_envs)}")
                        print(f"    (Failed environments assigned {max_steps} steps)")
                else:
                    print(f"    No per-environment data available")
                    if 'mean_completion_steps' in result:
                        print(f"    Overall mean: {result['mean_completion_steps']:.1f} steps")
                
                print()
        else:
            print(f"  No raw data available for {method}")
        
        print()

=== Completion Steps by Environment and Method ===

Method: Vanilla RL
--------------------------------------------------
  Run 1:
    Completed environments (0/20):
[]
    Failed environments (20/20): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
    (Failed environments assigned 300 steps)


Method: AV-WBFO w RL
--------------------------------------------------
  Run 1:
    Completed environments (20/20):
[98, 106, 118, 121, 126, 133, 135, 141, 145, 146, 151, 168, 185, 186, 191, 207, 207, 211, 245, 281]


Method: AV-WBFO w/o RL
--------------------------------------------------
  Run 1:
    Completed environments (15/20):
[117, 126, 185, 204, 210, 216, 221, 222, 245, 257, 258, 260, 280, 291, 299]
    Failed environments (5/20): [1, 5, 11, 13, 17]
    (Failed environments assigned 300 steps)


Method: MPPI w RL
--------------------------------------------------
  Run 1:
    Completed environments (20/20):
[135, 151, 163, 166, 177, 178, 179, 184, 186, 192, 201

Notes:

- If the notebook cannot find experiment folders, set the variables `franka_dir` and `robot_dir` manually to the correct experiment output directories and re-run the plotting cells.
- Figures are saved with names prefixed by `paper_` in each experiment directory.